<a href="https://colab.research.google.com/github/treerootboy/AIMetaCharacter/blob/master/%E5%B7%A5%E4%BD%9C/%E4%B8%AD%E5%8F%A4%E8%BD%A6/tc_item_update_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TC更新流失用户模型

# 资料
流失数据 
../input/item-update-churn
数据描述

数据时间：2022.11.2
付费更新：老券（通用券）+新券（买赠券）+支付点数 都算付费更新

只包含车商数据
1. 用户基本信息，注册时间、加入时长、流失时间、账号类型、县市、性别
2. 用户消费情况，储值金额、各商品消费金额、消费频次、连续消费时长
3. 用户标签，豪华厂牌，普通厂牌
3. 用户更新消费情况，加值卡储值、退款、消费、连续消费时常

## 数据处理规划
1. 清洗数据
> 完全合一
完整性：是否存在na，字段是否完整
全面性：观察一列全部数据，用常识判断是否合理，数据定义、数据本身、数据单位等
合法性：数据类型、内容、大小合法性，譬如性别存在未知，年龄超过150等
唯一性：是否存在重复数据
    - 填入 na 数据 => df.fillna(0)
    - 统一数据枚举 => df['field'].replace('old','new')
2. 分类数据，按各维度展示，流失/非流失
    - 多维度同时展示 => [pandas 实现](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#using-layout-and-targeting-multiple-axes)
3. 将高相关性的维度组合，了解维度关系

4. 进一步归一化、二值化数据维度
5. 拆分训练和验证数据集
6. 训练模型，观察数据结果


In [ ]:
#@title 数据表单设定{ run: "auto" }
#@description先设定表单，再按执行全部；数据周期2022.11.5往前推90天
start_date = "2022-10-01" #@param {type:"date"}
end_date = "2022-10-31" #@param {type:"date"}


# 数据处理

In [ ]:
# 加载包
import pandas as pd
from sklearn import preprocessing
import numpy as np
import seaborn as sb

!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
import matplotlib.pyplot as plt 
from matplotlib.font_manager import fontManager
from matplotlib.pyplot import MultipleLocator

# 改style要在改font之前
# plt.style.use('seaborn')  

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title
df = pd.read_csv('/content/drive/MyDrive/8891数据/2022更新流失预测模型/流失特征-三次清洗.csv', parse_dates=[0,1,2,3,4])
df.rename(columns={
    "會員註冊時間": "regist_date",
    "開始觀察時間（90天前）": "before_last_payment_90days",
    "流失時間": "last_payment_date",
    "年月日": "date",
    "最早更新時間": "first_update_date",
    "會員角色": "role",
    "是否嚴選車商（清洗）": "is_selection_member",
    "縣市": "region",
    "性別": "sex",
    "是否流失（清洗）": "churn",
    "成交物件數": "deal_item_num",
    "儲值": "store_amount",
    "點擊數": "item_clicks",
    "點擊轉化率": "item_clicks_rate",
    "更新-認列營收（稅前）": "item_update_revenue_without_tax",
    "更新次數": "item_update_count",
    "更新物件數": "updated_items",
    "豪華廠牌-物件數": "exbrand_items",
    "加入時長（月）": "join_months",
    "加值卡儲值": "item_update_store",
    "加值卡退款": "item_update_refund",
    "加值卡消費": "item_update_cost",
    "加值卡餘額": "item_update_balance",
    "精選-認列營收（稅前）": "jx_cost",
    "刊登物件數": "items",
    "冷門廠牌-物件數": "cobrand_items",
    "聯絡點擊轉化率": "contact_rate",
    "聯絡數": "contact_clicks",
    "流失當天物件數": "churn_date_items",
    "流失天數（最後一次使用距今天數）": "last_payment_diffdate",
    "瀏覽數": "views",
    "賣家id": "m_id",
    "普通廠牌-物件數": "nobrand_items",
    "曝光數": "impressions",
    "餘額": "balance",
    "置頂-認列營收（稅前）": "topitem_cost",
    "中古車加值-認列營收（稅前）": "extra_cost",
    "中古車刊登-認列營收（稅前）": "item_post_cost",
    "總認列營收（稅前）": "total_cost"
}, inplace=True)
df

问题：由于流失时间不一样，需要统一时间节点，用相对时间进行评估
数据处理目标：所有车商，流失前30天数据

## 取2022年10月数据评估


In [ ]:
df = df.query('date >= "%s" and date <= "%s"' %(start_date, end_date))
df

## 合法性



In [ ]:
df.info()

上面格式看到部分格式不对，应为数值，但格式却是object，查看数据里面存在千分号，需要去掉数值千分号

In [ ]:
df[["store_amount","item_clicks","item_update_revenue_without_tax","exbrand_items","item_update_store","item_update_refund","item_update_cost","item_update_balance","jx_cost","contact_rate","views","nobrand_items","impressions","balance","topitem_cost","extra_cost","item_post_cost","total_cost"]] = df[["store_amount","item_clicks","item_update_revenue_without_tax","exbrand_items","item_update_store","item_update_refund","item_update_cost","item_update_balance","jx_cost","contact_rate","views","nobrand_items","impressions","balance","topitem_cost","extra_cost","item_post_cost","total_cost"]].apply(lambda row: row.replace(',', '', regex=True).replace('∞', 0, regex=True).astype('float64'))

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
#@title
df.fillna(0, inplace=True)

In [ ]:
#@title
df.isna().sum()

In [ ]:
#@title
df.describe()

In [ ]:
#@title
df['m_id'].value_counts().count()

增加付费时长 payment_days

In [ ]:
#@title
df['payment_days'] = ((df['last_payment_date']-df['first_update_date']) / np.timedelta64(1, 'D')).astype(int)
df['payment_days']

将加值卡消费转正数

In [ ]:
#@title
df['item_update_cost'] = -df['item_update_cost']
df['item_update_cost']

# 数据分析

**将数据聚合到账号下(m_id)，以账号维度分析**

In [ ]:
#@title
midf = df.groupby('m_id').agg({
    # 用户信息
  "churn": 'last',
  "sex": 'last',
  "region": 'last',
  "is_selection_member": 'last',

  # 时间
  "join_months": 'last',
  "payment_days": 'last',
  "last_payment_diffdate": 'last',

  # 物件使用&效果
  "items": 'mean',
  "churn_date_items": 'last',
  "exbrand_items": 'last',
  "nobrand_items": 'last',
  "cobrand_items": 'last',
  "deal_item_num": 'sum',
  "views": 'mean',
  "impressions": 'mean',
  "item_clicks": 'mean',
  "item_clicks_rate": 'mean',
  "contact_rate": 'mean',
  "contact_clicks": 'mean',

  # 更新使用&消费情况
  "item_update_count": 'mean',
  "updated_items": 'mean',
  "item_update_revenue_without_tax": 'sum',
  "balance": 'last',
  "store_amount": 'sum',
  "item_update_store": 'sum',
  "item_update_refund": 'sum',
  "item_update_cost": 'sum',
  "item_update_balance": 'last',

  # 其他使用情况
  "jx_cost": 'sum',
  "topitem_cost": 'sum',
  "extra_cost": 'sum',
  "item_post_cost": 'sum',
  "total_cost": 'sum',
})

## 总体用户数据概览

### 用户信息

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 6))
midf['churn'].hist(ax=axes[0,0])
midf['is_selection_member'].hist(ax=axes[0,1])
midf['sex'].value_counts().plot(kind='bar', ax=axes[1,0])
midf['region'].value_counts().plot(kind='bar', ax=axes[1,1])
plt.show()


**从数据可以了解什么？**
1. 每月流失比例较少
2. 严选更新占比较少
3. 使用人群男性占大多数，但女性也占到接近 20%
4. 桃园车商更热衷使用更新，使用人数接近后面的2倍。使用用户集中在北部（桃园、新北、台北）

In [ ]:
#@title
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
tmpdf = midf.query('churn==0')[["join_months","payment_days", "items"]]
tmpdf.hist(ax=axes, bins=20)
axes[2].xaxis.set_major_locator(MultipleLocator(10))
plt.show()

In [ ]:
tmpdf.describe()

**从数据可以了解什么？**
1. 使用人群集中在 3 年，5-8年，11-14年。使用最多是1.5年内用户
2. 持续支付周期最多是5个月，说明大部分人都接纳更新并持续使用

### 消费情况

In [ ]:
#@title
tmpdf = midf[["total_cost","item_update_revenue_without_tax","jx_cost","topitem_cost","item_post_cost", "payment_days"]]
tmpdf.describe()

In [ ]:
#@title
fig, axes = plt.subplots(2,3,figsize=(30,10))
# churndf['item_update_cost'].hist(ax=axes[0,1], xrot=90)
# axes[0,1].xaxis.set_major_locator(MultipleLocator(5000))
# churndf['item_update_cost'].hist(ax=axes[0,1], xrot=90)
tmpdf.hist(ax=axes, xrot=90, bins=20)
axes[0,0].set_title('总消费')
axes[0,1].set_title('更新消费')
axes[0,1].xaxis.set_major_locator(MultipleLocator(2000))
axes[0,2].set_title('精选消费')
axes[0,2].xaxis.set_major_locator(MultipleLocator(2000))
axes[1,0].set_title('置顶消费')
axes[1,1].set_title('刊登消费')
axes[1,2].set_title('更新付费时长')
plt.suptitle('用户消费情况')
plt.show()

**从数据可以了解什么？**
1. 认列ARPU在1万左右，更新消费50%（消费，会有误差），精选10%，置顶12%，刊登30%
2. 认列ARPU， 人均:9571, 更新：5247，精选：1093，置顶：1205，刊登：3086，更新付费时长：134



In [ ]:
(midf['topitem_cost'] == 0).value_counts().plot(kind="bar")
plt.show()

## 流失用户分析

In [ ]:
churndf = midf.query('churn==1')

In [ ]:
churndf['churn'].count()

10月流失61人

In [ ]:
#@title
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
churndf['churn'].value_counts().plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('流失')
churndf['is_selection_member'].value_counts().plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('是否严选')
churndf['sex'].value_counts().plot(kind='bar', ax=axes[0,2])
axes[0,2].set_title('性别')
churndf['region'].value_counts().plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('地区')
churndf['join_months'].hist(ax=axes[1,1])
axes[1,1].set_title('加入时长')
churndf['join_months'].hist(ax=axes[1,2])
axes[1,2].set_title('更新消费时长')
plt.suptitle("流失车商用户信息情况")
plt.show()

In [ ]:
tmpdf = midf.query('churn==0 and region=="台北市"')
fig, axes = plt.subplots(1, 2, figsize=(15, 3))

axes[0].set_title('刊登消费')
tmpdf['item_post_cost'].hist(ax=axes[0], xrot=90)


axes[1].set_title('加值消费')
axes[1].set_xlim([0, 50000]) # 排除极端值, 少数超过5万
axes[1].xaxis.set_major_locator(MultipleLocator(2000))
tmpdf['extra_cost'].hist(ax=axes[1], xrot=90, bins=100)

plt.suptitle("台北市未流失车商消费")
plt.show()

**从数据可以了解什么？**
1. 流失用户中，严选、性别、地区比例基本与使用人群相符，没有太大差别
2. 其中流失地区最多仍是桃园，但占比较多的台北，流失反而较少。台北的车商更愿意使用更新服务。
3. 台北车商更愿意用，并非他们更愿意花钱，相反他们的消费都在 4000 内

In [ ]:
tmpdf = churndf[["total_cost","item_update_revenue_without_tax","jx_cost","topitem_cost","item_post_cost", "payment_days"]]
fig, axes = plt.subplots(2,3,figsize=(20,10))
# churndf['total_cost'].hist(ax=axes[0,0], bins=200, xrot=90)
# axes[0,0].xaxis.set_major_locator(MultipleLocator(5000))
# churndf['item_update_cost'].hist(ax=axes[0,1], xrot=90)
# axes[0,1].xaxis.set_major_locator(MultipleLocator(5000))
# churndf['item_update_cost'].hist(ax=axes[0,1], xrot=90)
tmpdf.hist(ax=axes)
axes[0,0].set_title('总认列')
axes[0,1].set_title('更新认列')
axes[0,2].set_title('精选认列')
axes[1,0].set_title('置顶认列')
axes[1,1].set_title('刊登认列')
axes[1,2].set_title('更新付费时长')
plt.suptitle('流失用户认列情况')
plt.show()

In [ ]:
tmpdf.mean()

**从数据可以了解什么？**
1. 流失用户虽然是车商，但都是小额消费，刊登认列基本在500以下，但更新的LT挺长在3个月+，可能购买了8笔的季度套餐
2. 重更新，轻置顶等大额消费

这类用户猜测是对价格敏感的小型车商

In [ ]:
tmpdf = churndf[["item_update_count","updated_items","item_update_revenue_without_tax","balance","store_amount","item_update_store","item_update_refund","item_update_cost","item_update_balance"]]
fig, axes = plt.subplots(3,3,figsize=(20,10))
tmpdf.hist(ax=axes, bins=20, xrot=90)
axes[0,0].set_title('更新次数')
axes[0,1].set_title('更新物件数')
axes[0,1].xaxis.set_major_locator(MultipleLocator(5))
axes[0,2].set_title('更新认列')
axes[1,0].set_title('点数余额')
axes[1,1].set_title('储值金额')
axes[1,1].xaxis.set_major_locator(MultipleLocator(500))
axes[1,2].set_title('加值卡储值')
axes[1,2].xaxis.set_major_locator(MultipleLocator(50))
axes[2,0].set_title('加值卡退款')
axes[2,1].set_title('加值卡消费')
axes[2,2].set_title('加值卡余额')
axes[2,2].xaxis.set_major_locator(MultipleLocator(1000))
plt.suptitle('流失用户消费情况')
plt.show()

In [ ]:
tmpdf.describe()

🌟**从数据可以了解什么？**
1. 75%的流失用户都没有储值，余额在0。储值行为可以作为一个明显特征，用户的账户健康程度（有持续的流水），说明用户在持续使用产品。
2. 另外用户的刊登物件是否有缩减也可以作为一个明显特征。

In [ ]:
tmpdf = churndf[["items","churn_date_items","exbrand_items","nobrand_items","cobrand_items","deal_item_num"]]
fig, axes = plt.subplots(2,3,figsize=(20,10))
tmpdf.hist(ax=axes, bins=20, xrot=90)
axes[0,0].set_title('平均在售物件数')
axes[0,0].xaxis.set_major_locator(MultipleLocator(5))
axes[0,1].set_title('流失当天物件数')
axes[0,2].set_title('豪华厂牌物件数') # TODO: 标识用户是豪华为主还是普通为主
axes[1,0].set_title('普通厂牌物件数')
axes[1,1].set_title('冷门厂牌物件数')
axes[1,2].set_title('成交物件数')
axes[1,2].xaxis.set_major_locator(MultipleLocator(1))
plt.suptitle('流失用户物件情况')
plt.show()

In [ ]:
tmpdf.describe()

In [ ]:
churndf['main_brand'] = churndf[["exbrand_items","nobrand_items","cobrand_items"]].idxmax(axis=1).replace('exbrand_items','豪华厂牌').replace('nobrand_items','普通厂牌').replace('cobrand_items','冷门厂牌')


In [ ]:
churndf['main_brand'].hist()

**从数据可以了解什么？**
1. 流失用户刊登普通厂牌为主的较多
2. 流失用户几乎没有成交

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'newcar8891' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
bgdf = client.query("""
select m_id,
       count(distinct if(substr(note, 1, 5) = '加值卡扣除', 0, relate_data)) as use_coupon,
       count(distinct if(substr(note, 1, 5) = '加值卡扣除', relate_data, 0)) as use_point,
       count(distinct if(sale_code between 9005001 and 9005004, relate_data, 0)) as nbrand_updates,
       count(distinct if(sale_code between 9005011 and 9005014, relate_data, 0)) as exbrand_updates,
       count(distinct if(sale_code in (9005001, 9005011), relate_data, 0)) as plan_below_4,
       count(distinct if(sale_code in (9005002, 9005012), relate_data, 0)) as plan_below_9,
       count(distinct if(sale_code in (9005003, 9005013), relate_data, 0)) as plan_below_23,
       count(distinct if(sale_code in (9005004, 9005014), relate_data, 0)) as plan_below_24,
       count(distinct relate_data) as total_update_items,
       count(1) as total_updates
       from t8891.t_sele_item_update_bill where record_time between '2022-10-01' and '2022-11-01' and io_flag = '-' group by m_id;
""").to_dataframe()
bgdf

In [ ]:
bgdf.set_index('m_id', inplace=True)
midf.index = midf.index.str.replace(',','').astype('int64')

In [ ]:
mdf = midf.join(bgdf)
mdf

In [ ]:
tmpdf = mdf[["use_coupon","use_point","nbrand_updates","exbrand_updates","plan_below_4","plan_below_9","plan_below_23","plan_below_24","total_update_items","total_updates"]]
tmpdf.hist(figsize=(20,15), bins=20)
plt.show()